In [13]:
import os
import numpy as np
import cv2
import glob
from sklearn.model_selection import train_test_split


In [2]:
# Define directories
vv_dir = '../Final_database/vv/'
vh_dir = '../Final_database/vh/'
water_body_dir = '../Final_database/water_body_label/'
flood_label_dir = '../Final_database/flood_label/'
preprocessed_dir = '../preprocessed_data/'


In [3]:
def load_images_from_folder(folder, label_folder):
    # Initialiser les tableaux NumPy avec la taille requise
    num_images = len(glob.glob(os.path.join(folder, '*.png')))
    images = np.zeros((num_images, 256, 256), dtype=np.uint8)
    labels = np.zeros((num_images, 256, 256), dtype=np.uint8)

    for i, filename in enumerate(glob.glob(os.path.join(folder, '*.png'))):
        # Charger l'image en uint8
        img = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"Erreur de chargement de l'image: {filename}")
            continue  # Passer à l'image suivante
        
        img = cv2.resize(img, (256, 256))  # Redimensionner
        images[i] = img  # Remplir le tableau NumPy
        
        # Charger le label correspondant
        label_name = os.path.basename(filename).replace('_vv.png', '.png')
        label_path = os.path.join(label_folder, label_name)
        label_img = cv2.imread(label_path, cv2.IMREAD_GRAYSCALE)
        if label_img is None:
            print(f"Erreur de chargement du label: {label_path}")
            continue  # Passer au label suivant
        
        label_img = cv2.resize(label_img, (256, 256))
        binary_label = (label_img > 0).astype(np.uint8)
        labels[i] = binary_label  # Remplir le tableau NumPy

    # Retirer les images et labels qui n'ont pas pu être chargés
    valid_indices = np.where(images.sum(axis=(1, 2)) > 0)[0]
    return images[valid_indices], labels[valid_indices]

# Exemple d'utilisation



In [4]:
images, labels = load_images_from_folder(vv_dir,water_body_dir)

X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

In [5]:
import tensorflow as tf
from tensorflow.keras import layers, models

def unet_model_with_water_body(input_shape=(256, 256, 1)):
    """
    Build a U-Net model that takes in VV, VH polarization images and 
    the water body label as input to predict the flooded areas.
    
    Input: (VV, VH, Water Body Label) -> 3 channels
    Output: Flooded area segmentation -> 1 channel
    """

    inputs = layers.Input(shape=input_shape)

    # Encoder
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    # Decoder
    u1 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(p2)
    u1 = layers.concatenate([u1, c2])
    c3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u1)
    c3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c3)

    u2 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c3)
    u2 = layers.concatenate([u2, c1])
    c4 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u2)
    c4 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c4)

    # Output layer (flooded area prediction)
    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(c4)

    # Compile the model
    model = models.Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

# Example usage:
# The input shape is (256, 256, 3) where we have 3 channels (VV, VH, Water Body Label)
model = unet_model_with_water_body()
model.summary()


2024-10-26 16:25:58.698532: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-26 16:25:58.765440: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-26 16:25:58.783383: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-26 16:25:58.904199: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-26 16:26:00.201656: W tensorflow/compiler/tf2

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 256, 256,  │        640 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 256, 256,  │     36,928 │ conv2d[0][0]      │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 128, 128,  │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 128, 128,  │     73,856 │ max_pooling2d[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 128, 128,  │    147,584 │ conv2d_2[0][0]    │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 64, 64,    │          0 │ conv2d_3[0][0]    │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose    │ (None, 128, 128,  │     65,664 │ max_pooling2d_1[… │
│ (Conv2DTranspose)   │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 128, 128,  │          0 │ conv2d_transpose… │
│ (Concatenate)       │ 256)              │            │ conv2d_3[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 128, 128,  │    295,040 │ concatenate[0][0] │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 128, 128,  │    147,584 │ conv2d_4[0][0]    │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose_1  │ (None, 256, 256,  │     32,832 │ conv2d_5[0][0]    │
│ (Conv2DTranspose)   │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 256, 256,  │          0 │ conv2d_transpose… │
│ (Concatenate)       │ 128)              │            │ conv2d_1[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 256, 256,  │     73,792 │ concatenate_1[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_7 (Conv2D)   │ (None, 256, 256,  │     36,928 │ conv2d_6[0][0]    │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_8 (Conv2D)   │ (None, 256, 256,  │         65 │ conv2d_7[0][0]    │
│                     │ 1)                │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 910,913 (3.47 MB)

 Trainable params: 910,913 (3.47 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:
#### TRAINING #####

# Ensure the directory for saving model parameters exists
checkpoint_dir = 'model_parameters'
os.makedirs(checkpoint_dir, exist_ok=True)

# Define the checkpoint callback to save model parameters at specified epochs
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(checkpoint_dir, 'model_epoch_{epoch:02d}.weights.h5'),
    save_weights_only=True,
    save_freq='epoch',
)

# Train the U-Net model with the checkpoint callback
model.fit(X_train, y_train, epochs=20, batch_size=2, validation_data=(X_val, y_val), callbacks=[checkpoint_callback])

Epoch 1/20
570/570 ━━━━━━━━━━━━━━━━━━━━ 30s 52ms/step - accuracy: 0.9829 - loss: 0.0594 - val_accuracy: 0.9738 - val_loss: 0.0844
Epoch 2/20
570/570 ━━━━━━━━━━━━━━━━━━━━ 30s 53ms/step - accuracy: 0.9840 - loss: 0.0492 - val_accuracy: 0.9738 - val_loss: 0.0854
Epoch 3/20
570/570 ━━━━━━━━━━━━━━━━━━━━ 31s 55ms/step - accuracy: 0.9842 - loss: 0.0623 - val_accuracy: 0.9739 - val_loss: 0.0779
Epoch 4/20
570/570 ━━━━━━━━━━━━━━━━━━━━ 32s 55ms/step - accuracy: 0.9871 - loss: 0.0410 - val_accuracy: 0.9733 - val_loss: 0.2816
Epoch 5/20
570/570 ━━━━━━━━━━━━━━━━━━━━ 32s 56ms/step - accuracy: 0.9739 - loss: 0.1669 - val_accuracy: 0.9742 - val_loss: 0.0836
Epoch 6/20
570/570 ━━━━━━━━━━━━━━━━━━━━ 35s 61ms/step - accuracy: 0.9828 - loss: 0.0511 - val_accuracy: 0.9744 - val_loss: 0.0860
Epoch 7/20
570/570 ━━━━━━━━━━━━━━━━━━━━ 35s 62ms/step - accuracy: 0.9858 - loss: 0.0447 - val_accuracy: 0.9741 - val_loss: 0.0732
Epoch 8/20
570/570 ━━━━━━━━━━━━━━━━━━━━ 37s 64ms/step - accuracy: 0.9846 - loss: 0.0499 - 

In [10]:
##### PREDICTION DU MODELE #####
# After training, make predictions on a subset of the validation set (3 samples)
y_pred = model.predict(X_val[:10])
print(y_pred.shape)
y_temp = y_pred.reshape((10, 256, 256))

# Calculate the IoU metric for each prediction
iou_scores = []
for k in range(10):
    iou_score_image = 0
    for i in range(256):
        for j in range(256):
            binary_predictions = (y_temp[k][i][j] > 0.5).astype(int)
            if y_val[k][i][j] == binary_predictions:
                iou_score_image+=1
    iou_scores.append(iou_score_image/(256*256))

print("Average IoU score for 3 samples:", np.mean(iou_scores))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
(10, 256, 256, 1)
Average IoU score for 3 samples: 0.9986404418945313


In [11]:
##### IMPRIME LES IMAGES EN PNG ####
import imageio

# Ensure the output directory exists
output_dir = '../predicted_images'
os.makedirs(output_dir, exist_ok=True)

# Transform the 3 predicted images into PNG files
for i in range(10):
    # Rescale the image to 0-255 and convert to uint8
    img = (y_pred[i, :, :, 0] * 255).astype(np.uint8)
    # Save the image as a PNG file
    imageio.imwrite(os.path.join(output_dir, f'predicted_image_{i}.png'), img)